# Proyek Analisis Data: Air-quality-dataset
- **Nama:** Izzuddin Ahmad Afif
- **Email:** izzuddinafif@gmail.com
- **ID Dicoding:** izzuddinafif

## Menentukan Pertanyaan Bisnis


## Menentukan Pertanyaan Bisnis

- Bagaimana tren perubahan konsentrasi PM2.5 di masing-masing stasiun selama periode tertentu, dan faktor apa yang memengaruhinya?
- Faktor lingkungan atau aktivitas manusia apa yang memiliki korelasi signifikan dengan perbedaan kualitas udara antar stasiun?


## Import Semua Packages/Library yang Digunakan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

# Atur gaya plot secara default
plt.style.use('default')


## Data Wrangling

### Gathering Data

In [ ]:
# Load semua file CSV dalam folder 'csv'
folder_path = 'csv'
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Dictionary untuk menampung data tiap stasiun
station_dataframes = {}

# Memuat data dari setiap file CSV
for file in csv_files:
    base_name = os.path.basename(file)
    parts = base_name.split('_')
    if len(parts) > 3:
        station = parts[2]  # Nama stasiun
        df = pd.read_csv(file)
        station_dataframes[station] = df

# Tampilkan contoh data (hanya menampilkan 5 baris) dari satu stasiun sebagai sample
sample_station = list(station_dataframes.keys())[0] if station_dataframes else None
if sample_station:
    print(f"Data awal untuk stasiun {sample_station}:")
    display(station_dataframes[sample_station].head())
else:
    print("Tidak ditemukan file CSV dalam folder.")


**Insight:**
- Semua file CSV memiliki struktur yang seragam yang menunjukkan bahwa data dikumpulkan dengan format yang standar. Hal ini akan memudahkan langkah cleaning data dan analisis selanjutnya. 
- File CSV yang berbeda per stasiun yang kemudian dikelompokkan menggunakan dictionary memungkinkan analisis per stasiun secara terpisah maupun perbandingan antar stasiun. 
- Melihat kolom year, month, day, dan hour, kita bisa mengidentifikasi bahwa data merupakan data time series. Ini membuka peluang untuk analisis tren temporal (misalnya, tren harian, bulanan, atau tahunan).
- Struktur data yang sudah terpisah per stasiun memudahkan penerapan teknik analisis lanjutan seperti perbandingan kualitas udara antar stasiun, analisis geospasial, ataupun clustering berdasarkan pola-pola tertentu.

### Assessing Data

In [ ]:
def assess(df):
    results = {}
    
    # 1. Bentuk DataFrame
    results["shape"] = df.shape
    
    # 2. Missing Values
    results["missing_values"] = df.isnull().sum().to_dict()
    
    # 3. Nilai Invalid (cek apakah bisa dikonversi ke float)
    numeric_cols = ["PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "RAIN", "WSPM"]
    invalid_values = {}
    for col in numeric_cols:
        count = 0
        if col in df.columns:
            for val in df[col]:
                if pd.notnull(val):
                    try:
                        float(val)
                    except Exception:
                        count += 1
        invalid_values[col] = count
    results["invalid_values"] = invalid_values
    
    # 4. Baris duplikat
    results["duplicate_rows"] = int(df.duplicated().sum())
    
    # 5. Nilai tidak akurat (nilai negatif untuk kolom yang seharusnya non-negatif)
    non_negative_cols = ["PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "RAIN", "WSPM"]
    inaccurate_values = {}
    for col in non_negative_cols:
        count = 0
        if col in df.columns:
            for val in df[col]:
                try:
                    if float(val) < 0:
                        count += 1
                except Exception:
                    continue
        inaccurate_values[col] = count
    results["inaccurate_values"] = inaccurate_values
    
    # 6. Nilai tidak konsisten (untuk kolom non-numerik)
    inconsistent_values = {}
    for col in df.columns:
        if df[col].dtype == 'O':
            inconsistent_values[col] = df[col].dropna().unique().tolist()
    results["inconsistent_values"] = inconsistent_values
    
    # 7. Deteksi Outlier dengan metode IQR 
    outliers = {}
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    for col in numeric_columns:
        if df[col].nunique() <= 1:
            outliers[col] = 0
            continue
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[col] = sum(1 for x in df[col] if x < lower_bound or x > upper_bound)
    results["outliers"] = outliers
    
    return results

# Lakukan assessment untuk setiap stasiun dan kumpulkan ringkasan khusus untuk PM2.5
assessment_results = {}
summary_data = []

for station, df in station_dataframes.items():
    res = assess(df)
    assessment_results[station] = res
    summary_data.append({
       "Station": station,
       "Rows": res["shape"][0],
       "Missing_PM2.5": res["missing_values"].get("PM2.5", 0),
       "Invalid_PM2.5": res["invalid_values"].get("PM2.5", 0),
       "Duplicate_Rows": res["duplicate_rows"],
       "Outliers_PM2.5": res["outliers"].get("PM2.5", 0)
    })

summary_df = pd.DataFrame(summary_data)
print("Ringkasan Assessment untuk PM2.5 per stasiun:")
display(summary_df)

# Visualisasi sederhana: Missing values & Outlier PM2.5 per stasiun
plt.figure(figsize=(10, 6))
plt.bar(summary_df["Station"], summary_df["Missing_PM2.5"], color="skyblue")
plt.xlabel("Stasiun")
plt.ylabel("Jumlah Missing PM2.5")
plt.title("Missing PM2.5 per Stasiun")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
plt.bar(summary_df["Station"], summary_df["Outliers_PM2.5"], color="salmon")
plt.xlabel("Stasiun")
plt.ylabel("Jumlah Outlier PM2.5")
plt.title("Outlier PM2.5 per Stasiun")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


**Insight:**
- Semua file memiliki jumlah baris dan kolom yang sama (35.064 baris dan 18 kolom), yang menunjukkan bahwa data dikumpulkan dalam periode dan struktur yang konsisten antar stasiun.
- Jumlah nilai hilang pada kolom, terutama untuk PM2.5, bervariasi antar stasiun. Misalnya, ada stasiun yang memiliki missing value PM2.5 yang tinggi (seperti Shunyi dengan 913 nilai hilang) dibandingkan dengan stasiun lain (misalnya Wanliu dengan 382 nilai hilang). Ini mengindikasikan adanya variasi dalam kualitas pengumpulan data atau masalah pencatatan di masing-masing lokasi.
- Tidak ditemukan baris duplikat dan invalid pada data.
- Deteksi outlier menggunakan metode IQR menunjukkan adanya jumlah pencilan yang cukup tinggi pada kolom PM2.5 (misalnya 1626 outlier di Huairou).

### Cleaning Data

In [ ]:
# Fungsi untuk imputasi outlier menggunakan metode IQR
def impute_outliers(s, lower_quantile=0.25, upper_quantile=0.75, k=1.5):
    Q1 = s.quantile(lower_quantile)
    Q3 = s.quantile(upper_quantile)
    IQR = Q3 - Q1
    lower_bound = Q1 - k * IQR
    upper_bound = Q3 + k * IQR
    median_val = s.median()

    s_imputed = s.copy()
    s_imputed[s < lower_bound] = median_val
    s_imputed[s > upper_bound] = median_val
    return s_imputed

# Daftar kolom yang akan dibersihkan
columns_to_clean = ["PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "RAIN", "WSPM"]

# Proses cleaning dan imputasi untuk setiap stasiun
cleaned_dataframes = {}
stats_summary = []

for station, df in station_dataframes.items():
    # 1. Gabungkan kolom tanggal menjadi datetime
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
    df.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
    
    # 2. Pindahkan kolom datetime ke posisi kedua
    datetime_col = df.pop('datetime')
    df.insert(1, 'datetime', datetime_col)

    # 3. Pastikan semua kolom numerik
    for col in columns_to_clean:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # 4. Imputasi Missing Values (NaN)
    df.sort_values('datetime', inplace=True)

    for col in columns_to_clean:
        df[f"{col}_clean"] = df[col].interpolate(method='linear')  # Interpolasi linier
        df[f"{col}_clean"] = df[f"{col}_clean"].ffill().bfill()  # FFill & BFill
        df[f"{col}_clean"] = df[f"{col}_clean"].fillna(df[f"{col}_clean"].mean())  # Isi NaN dengan rata-rata

    # 5. Imputasi Outlier menggunakan metode IQR
    for col in columns_to_clean:
        df[f"{col}_imputed"] = impute_outliers(df[f"{col}_clean"])

    # Simpan hasil cleaning ke dictionary
    cleaned_dataframes[station] = df

    # 6. Statistik ringkasan sebelum & sesudah cleaning
    stats_summary.append({
        "Station": station,
        "Original_PM2.5_Mean": df["PM2.5"].mean(),
        "Clean_PM2.5_Mean": df["PM2.5_clean"].mean(),
        "Imputed_PM2.5_Mean": df["PM2.5_imputed"].mean(),
        "Original_PM2.5_Median": df["PM2.5"].median(),
        "Imputed_PM2.5_Median": df["PM2.5_imputed"].median(),
        "Missing_Values_After_Cleaning": df[columns_to_clean].isna().sum().sum(),  # Cek apakah masih ada NaN
    })

# 7. Menampilkan Ringkasan Cleaning Data
stats_df = pd.DataFrame(stats_summary)
print("Summary Statistik PM2.5 per Stasiun:")
display(stats_df)


**Insight:**
- Dengan menggabungkan kolom tanggal (year, month, day, hour) menjadi satu kolom datetime dan mengurutkan data berdasarkan waktu, kita mendapatkan deret waktu yang lebih konsisten. Hal ini memudahkan analisis tren PM2.5 secara time series.
- Penggunaan interpolasi linier untuk mengisi missing data membuat data PM2.5 menjadi lebih kontinu. Jika missing data tidak diisi, bisa terjadi celah yang mengganggu analisis tren. Dengan imputasi, distribusi data menjadi lebih lengkap.
- Dari statistik awal, nilai rata-rata (mean) PM2.5 cenderung lebih tinggi karena adanya nilai ekstrim. Setelah dilakukan imputasi outlier dengan mengganti nilai ekstrim menggunakan median, nilai mean menurun secara signifikan, sedangkan median tetap stabil. Ini menunjukkan bahwa outlier sebelumnya memberikan bias pada perhitungan rata-rata.
- Setelah proses cleaning, data PM2.5 menjadi lebih homogen dan siap untuk analisis tren serta perbandingan kualitas udara antar stasiun. Data yang sudah dibersihkan memberikan dasar yang lebih solid untuk menjawab pertanyaan bisnis, misalnya apakah terdapat tren penurunan atau kenaikan konsentrasi PM2.5 dan perbedaan antar stasiun.



## Exploratory Data Analysis (EDA)

In [ ]:
# --- EDA: Eksplorasi Data Umum ---

# 1. Tampilkan contoh data (5 baris) dari setiap stasiun setelah cleaning
print("Contoh data (5 baris) dari setiap stasiun setelah cleaning:")
for station, df in cleaned_dataframes.items():
    print(f"\nStasiun: {station}")
    display(df.head(5))

# 2. Kelompokkan stasiun ke dalam kategori wilayah yang lebih luas
station_categories = {
    "Aotizhongxin": "Perkotaan",
    "Dongsi": "Perkotaan",
    "Guanyuan": "Perkotaan",
    "Wanshouxigong": "Perkotaan",
    "Gucheng": "Industri",
    "Changping": "Pinggiran/Suburban",
    "Huairou": "Pinggiran/Suburban",
    "Shunyi": "Pinggiran/Suburban",
    "Dingling": "Sejarah & Budaya",
    "Tiantan": "Sejarah & Budaya",
    "Wanliu": "Pendidikan & Penelitian",
    "Nongzhanguan": "Pendidikan & Penelitian"
}

# Tambahkan kategori wilayah ke masing-masing DataFrame
for station, df in cleaned_dataframes.items():
    df["Kategori"] = station_categories.get(station, "Tidak Dikategorikan")

# 3. Ringkasan Statistik PM2.5 dan Faktor Lingkungan untuk Semua Stasiun
global_summary = []
for station, df in cleaned_dataframes.items():
    # Ambil statistik deskriptif untuk semua variabel yang telah di-impute
    stats = df[["PM2.5_imputed", "PM10_imputed", "SO2_imputed", "NO2_imputed", 
                "CO_imputed", "O3_imputed", "TEMP_imputed", "PRES_imputed", 
                "DEWP_imputed", "WSPM_imputed"]].describe().T  # Transpose agar statistik menjadi baris
    stats["Station"] = station  # Tambahkan nama stasiun sebagai kolom
    stats["Kategori"] = station_categories[station]  # Tambahkan kategori wilayah
    global_summary.append(stats)

# Gabungkan semua ringkasan menjadi satu DataFrame
summary_df2 = pd.concat(global_summary).reset_index().rename(columns={"index": "Variable"})
print("Ringkasan Statistik PM2.5 dan Faktor Lingkungan per Stasiun:")
display(summary_df2)

# 4. Agregasi Harian, Bulanan, dan Tahunan untuk Tren PM2.5 & Faktor Lingkungan
agg_list_pm25 = []
agg_list_factors = []

for station, df in cleaned_dataframes.items():
    df["date"] = df["datetime"].dt.date
    df["year"] = df["datetime"].dt.year
    df["month"] = df["datetime"].dt.month
    
    # Agregasi PM2.5
    daily_avg = df.groupby("date")["PM2.5_imputed"].mean().reset_index()
    daily_avg["Station"] = station
    
    monthly_avg = df.groupby(["year", "month"])["PM2.5_imputed"].mean().reset_index()
    monthly_avg["Station"] = station
    
    yearly_avg = df.groupby("year")["PM2.5_imputed"].mean().reset_index()
    yearly_avg["Station"] = station
    
    agg_list_pm25.append((daily_avg, monthly_avg, yearly_avg))
    
    # Agregasi Faktor Lingkungan
    monthly_factors = df.groupby(["year", "month"])[["SO2_imputed", "NO2_imputed", "CO_imputed", 
                                                      "O3_imputed", "TEMP_imputed", "PRES_imputed", 
                                                      "DEWP_imputed", "WSPM_imputed"]].mean().reset_index()
    monthly_factors["Station"] = station
    
    yearly_factors = df.groupby("year")[["SO2_imputed", "NO2_imputed", "CO_imputed", 
                                          "O3_imputed", "TEMP_imputed", "PRES_imputed", 
                                          "DEWP_imputed", "WSPM_imputed"]].mean().reset_index()
    yearly_factors["Station"] = station

    agg_list_factors.append((monthly_factors, yearly_factors))

# 5. Agregasi Tambahan Berdasarkan Kategori Wilayah
all_data = []
for station, df in cleaned_dataframes.items():
    temp_df = df.copy()
    temp_df["Kategori"] = station_categories.get(station, "Tidak Dikategorikan")  # Pastikan kategori ditambahkan
    all_data.append(temp_df)

all_data_df = pd.concat(all_data)

# Tampilkan ringkasan data setelah dikategorikan
print("Ringkasan data setelah penambahan kategori wilayah:")
display(all_data_df.groupby("Kategori")[["PM2.5_imputed", "PM10_imputed", "SO2_imputed", "NO2_imputed", 
                                         "CO_imputed", "O3_imputed", "TEMP_imputed", "PRES_imputed", 
                                         "DEWP_imputed", "WSPM_imputed"]].mean().reset_index())


**Insight**  

- **Tampilan Data Awal & Statistik Deskriptif:**  
  - Tampilan 5 baris pertama dari setiap stasiun menunjukkan bahwa struktur data telah bersih dan konsisten setelah proses cleaning, termasuk format datetime serta kolom polutan dan faktor lingkungan.  
  - Penambahan kategori wilayah (Perkotaan, Industri, Suburban, Sejarah & Budaya, serta Pendidikan & Penelitian) memungkinkan analisis lebih lanjut mengenai hubungan antara lokasi dan kualitas udara.  
  - Ringkasan statistik menunjukkan adanya perbedaan signifikan antar stasiun dalam hal konsentrasi polutan dan faktor lingkungan, yang dapat memberikan indikasi pengaruh lokal terhadap kualitas udara.  
  - Konsentrasi PM2.5 yang lebih tinggi di beberapa kategori wilayah seperti industri dan perkotaan mengindikasikan adanya faktor eksternal seperti aktivitas kendaraan, industri, atau kepadatan populasi yang dapat memengaruhi kualitas udara.  

- **Agregasi Harian, Bulanan, dan Tahunan:**  
  - Agregasi data berdasarkan waktu memberikan gambaran mengenai pola perubahan kualitas udara dalam jangka pendek dan jangka panjang.  
  - Agregasi harian menunjukkan variasi fluktuasi PM2.5 dalam satu hari, yang kemungkinan dipengaruhi oleh faktor-faktor seperti pola lalu lintas dan kondisi atmosfer.  
  - Agregasi bulanan memperlihatkan pola musiman di mana polusi udara cenderung meningkat pada bulan-bulan musim dingin dan menurun selama musim panas.  
  - Agregasi tahunan berguna untuk memahami tren polusi udara dari waktu ke waktu, apakah mengalami peningkatan atau penurunan secara keseluruhan.  

- **Perbedaan PM2.5 Berdasarkan Kategori Wilayah:**  
  - Rata-rata PM2.5 tertinggi ditemukan di kategori industri, diikuti oleh kawasan perkotaan, yang menunjukkan bahwa aktivitas manusia di daerah ini memiliki dampak besar terhadap tingkat polusi udara.  
  - Wilayah pinggiran atau suburban memiliki tingkat PM2.5 yang lebih rendah dibandingkan dengan kawasan perkotaan dan industri, mengindikasikan bahwa faktor lingkungan seperti vegetasi dan jarak dari sumber polusi utama berperan dalam meningkatkan kualitas udara.  
  - Kategori sejarah & budaya serta pendidikan & penelitian cenderung memiliki tingkat PM2.5 lebih rendah dibandingkan perkotaan, meskipun masih lebih tinggi dibandingkan daerah pinggiran, yang mungkin dipengaruhi oleh lalu lintas wisatawan atau kendaraan pribadi di sekitar area tersebut.  

- **Validasi Konsistensi Data:**  
  - Data telah bersih dengan tidak adanya nilai NaN atau anomali besar dalam distribusi setelah dilakukan proses imputasi dan pengelompokan kategori wilayah.  
  - Dengan adanya agregasi dan klasifikasi wilayah, data telah siap untuk tahap analisis lebih lanjut guna menjawab pertanyaan bisnis terkait faktor lingkungan dan perbedaan kualitas udara antar lokasi.

## Visualization & Explanatory Analysis

### Pertanyaan 1:

In [ ]:
# Visualisasi Tren PM2.5 Bulanan per Stasiun
plt.figure(figsize=(12, 6))

for _, monthly_avg, _ in agg_list_pm25:
    monthly_avg["Year-Month"] = pd.to_datetime(monthly_avg["year"].astype(str) + "-" + monthly_avg["month"].astype(str))  # Format datetime
    plt.plot(monthly_avg["Year-Month"], monthly_avg["PM2.5_imputed"], marker='o', label=monthly_avg["Station"].iloc[0])

plt.xlabel("Bulan-Tahun")
plt.ylabel("Rata-rata PM2.5")
plt.title("Tren Bulanan Rata-rata PM2.5 per Stasiun")
plt.xticks(rotation=45)  
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="Stasiun")
plt.grid(True, linestyle="--", alpha=0.7)  
plt.tight_layout()
plt.show()

# Visualisasi Tren PM2.5 Bulanan Berdasarkan Kategori Wilayah
plt.figure(figsize=(12, 6))

for kategori, df in all_data_df.groupby("Kategori"):
    monthly_trend = df.groupby(["year", "month"])["PM2.5_imputed"].mean().reset_index()
    monthly_trend["Year-Month"] = pd.to_datetime(monthly_trend["year"].astype(str) + "-" + monthly_trend["month"].astype(str))
    plt.plot(monthly_trend["Year-Month"], monthly_trend["PM2.5_imputed"], marker='o', label=kategori)

plt.xlabel("Bulan-Tahun")
plt.ylabel("Rata-rata PM2.5")
plt.title("Tren Bulanan Rata-rata PM2.5 Berdasarkan Kategori Wilayah")
plt.xticks(rotation=45)
plt.legend(title="Kategori Wilayah", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# Visualisasi Rata-rata PM2.5 per Bulan dalam Setahun untuk Semua Stasiun
plt.figure(figsize=(10, 6))

monthly_avg_all = all_data_df.groupby("month")["PM2.5_imputed"].mean().reset_index()
plt.plot(monthly_avg_all["month"], monthly_avg_all["PM2.5_imputed"], marker='o', linestyle='-', label="Rata-rata Semua Stasiun")

plt.xlabel("Bulan")
plt.ylabel("Rata-rata PM2.5")
plt.title("Rata-rata PM2.5 per Bulan dalam Setahun (Semua Stasiun)")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mar", "Apr", "Mei", "Jun", "Jul", "Agu", "Sep", "Okt", "Nov", "Des"])
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend()
plt.show()


# Visualisasi Rata-rata PM2.5 per Bulan Berdasarkan Kategori Wilayah
plt.figure(figsize=(12, 6))

for kategori, df in all_data_df.groupby("Kategori"):
    monthly_avg = df.groupby("month")["PM2.5_imputed"].mean().reset_index()
    plt.plot(monthly_avg["month"], monthly_avg["PM2.5_imputed"], marker='o', linestyle='-', label=kategori)

plt.xlabel("Bulan")
plt.ylabel("Rata-rata PM2.5")
plt.title("Rata-rata PM2.5 per Bulan Berdasarkan Kategori Wilayah")
plt.xticks(range(1, 13), ["Jan", "Feb", "Mar", "Apr", "Mei", "Jun", "Jul", "Agu", "Sep", "Okt", "Nov", "Des"])
plt.legend(title="Kategori Wilayah", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()


**Insight**  

- **Tren Musiman PM2.5 yang Konsisten**  
  - Grafik menunjukkan pola musiman yang jelas, di mana konsentrasi PM2.5 cenderung meningkat pada akhir tahun hingga awal tahun berikutnya dan menurun selama pertengahan tahun.  
  - Tren ini terlihat konsisten di seluruh stasiun dan kategori wilayah, yang menunjukkan adanya faktor musiman yang berpengaruh besar terhadap kualitas udara.  
  - Musim dingin memiliki tingkat PM2.5 yang lebih tinggi, kemungkinan karena kondisi atmosfer yang stabil, suhu yang lebih rendah, dan peningkatan aktivitas pemanasan rumah tangga atau industri yang menggunakan bahan bakar fosil.  

- **Perbedaan Konsentrasi PM2.5 Antar Wilayah**  
  - Wilayah industri dan perkotaan menunjukkan tingkat PM2.5 yang lebih tinggi dibandingkan dengan wilayah pinggiran, sejarah, dan pendidikan.  
  - Ini menunjukkan bahwa aktivitas manusia, seperti transportasi, industri, dan kepadatan populasi, berkontribusi besar terhadap kualitas udara yang lebih buruk di area tersebut.  
  - Wilayah pinggiran dan sejarah cenderung memiliki tingkat polusi yang lebih rendah, kemungkinan karena lebih banyak ruang hijau dan lebih sedikit sumber emisi kendaraan atau industri.  

- **Pola Fluktuasi Bulanan PM2.5**  
  - Grafik rata-rata PM2.5 per bulan dalam satu tahun menunjukkan bahwa konsentrasi polusi udara mencapai puncaknya pada bulan-bulan musim dingin, seperti November hingga Maret, dan mengalami penurunan signifikan pada bulan musim panas seperti Juli dan Agustus.  
  - Faktor atmosferik seperti curah hujan yang lebih tinggi dan peningkatan sirkulasi udara pada musim panas dapat membantu mengencerkan polusi udara, sehingga menurunkan kadar PM2.5.  

- **Implikasi untuk Kebijakan dan Intervensi**  
  - Data ini memberikan wawasan yang penting bagi pembuat kebijakan untuk merancang strategi mitigasi polusi udara yang lebih efektif.  
  - Jika polusi udara meningkat pada bulan-bulan tertentu, langkah-langkah intervensi seperti pembatasan kendaraan, pengurangan emisi industri, atau peningkatan penghijauan dapat difokuskan pada periode tersebut.  
  - Peningkatan kesadaran masyarakat dan kebijakan berbasis musim dapat membantu mengurangi dampak buruk polusi udara pada kesehatan masyarakat.  

### Pertanyaan 2:

In [ ]:
# --- Pertanyaan 2: Faktor Lingkungan & Perbedaan Kualitas Udara Antar Stasiun ---

# Heatmap Korelasi Semua Stasiun
# Pilih hanya kolom numerik yang relevan untuk korelasi
relevant_cols = ["PM2.5_imputed", "PM10_imputed", "SO2_imputed", "NO2_imputed",
                 "CO_imputed", "O3_imputed", "TEMP_imputed", "PRES_imputed", 
                 "DEWP_imputed", "WSPM_imputed"]

numeric_df = all_data_df[relevant_cols]  # Ambil hanya kolom yang relevan

# Hitung matriks korelasi
corr_matrix = numeric_df.corr()

# Tampilkan heatmap korelasi
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriks Korelasi - Semua Stasiun (Bersih)")
plt.tight_layout()
plt.show()



# Faktor lingkungan dengan korelasi lebih signifikan terhadap PM2.5
factors = ["DEWP_imputed", "WSPM_imputed"]  # Fokus hanya pada faktor yang lebih relevan

fig, axes = plt.subplots(1, 2, figsize=(14, 6))  # 1 baris, 2 kolom
axes = axes.flatten()

for i, factor in enumerate(factors):
    axes[i].scatter(all_data_df[factor], all_data_df["PM2.5_imputed"], alpha=0.3)
    axes[i].set_xlabel(factor)
    axes[i].set_ylabel("PM2.5")
    axes[i].set_title(f"Hubungan {factor} dengan PM2.5")
    axes[i].grid(True, linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()


# Bar Chart Rata-rata PM2.5 Berdasarkan Kategori Wilayah
pm25_per_kategori = all_data_df.groupby("Kategori")["PM2.5_imputed"].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x="Kategori", y="PM2.5_imputed", hue="Kategori", data=pm25_per_kategori, palette="coolwarm", legend=False)
plt.xticks(rotation=45)
plt.xlabel("Kategori Wilayah")
plt.ylabel("Rata-rata PM2.5")
plt.title("Rata-rata PM2.5 Berdasarkan Kategori Wilayah")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()



# Boxplot Distribusi PM2.5 Berdasarkan Kategori Wilayah
plt.figure(figsize=(12, 6))
sns.boxplot(x="Kategori", y="PM2.5_imputed", hue="Kategori", data=all_data_df, palette="coolwarm", legend=False)
plt.xticks(rotation=45)
plt.xlabel("Kategori Wilayah")
plt.ylabel("Distribusi PM2.5")
plt.title("Distribusi PM2.5 Berdasarkan Kategori Wilayah")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()



**Insight**  

- **Perbedaan Konsentrasi PM2.5 Berdasarkan Kategori Wilayah**  
  - Wilayah industri dan perkotaan memiliki konsentrasi PM2.5 yang lebih tinggi dibandingkan wilayah pinggiran/suburban dan sejarah & budaya.  
  - Aktivitas manusia, seperti transportasi dan industri, berkontribusi signifikan terhadap tingginya polusi udara.  
  - Wilayah pinggiran dan sejarah cenderung memiliki kualitas udara lebih baik, kemungkinan karena lebih sedikit kendaraan bermotor dan lebih banyak area hijau.  

- **Dampak Faktor Lingkungan terhadap PM2.5**  
  - Kecepatan angin (WSPM) memiliki korelasi negatif (-0.22) dengan PM2.5, menunjukkan bahwa angin kencang membantu menyebarkan polutan dan mengurangi konsentrasinya.  
  - Titik embun (DEWP) memiliki korelasi positif (0.20), yang menunjukkan bahwa kelembapan dapat mempengaruhi akumulasi polutan di udara.  
  - Suhu (TEMP) dan tekanan udara (PRES) memiliki korelasi yang sangat rendah dengan PM2.5 (-0.02 dan -0.05), yang menunjukkan bahwa perubahan faktor ini tidak berdampak signifikan terhadap polusi udara.  

- **Hubungan PM2.5 dengan Polutan Lain**  
  - PM10 memiliki korelasi tinggi dengan PM2.5 (0.78), yang menunjukkan bahwa keduanya kemungkinan berasal dari sumber yang sama, seperti emisi kendaraan dan industri.  
  - CO dan NO2 juga memiliki korelasi sedang dengan PM2.5 (0.62 dan 0.53), mengindikasikan bahwa transportasi memainkan peran utama dalam peningkatan polusi udara.  
  - SO2 memiliki korelasi lebih rendah (0.31), menunjukkan bahwa pembakaran batu bara dan aktivitas industri masih berkontribusi terhadap polusi udara, tetapi tidak sebesar sektor transportasi.

## Analisis Lanjutan (Opsional)

# Conclusion

1. **Tren Perubahan Konsentrasi PM2.5**  
   - Konsentrasi PM2.5 mengalami pola musiman yang jelas, dengan peningkatan signifikan pada akhir tahun hingga awal tahun berikutnya, dan penurunan selama pertengahan tahun.  
   - Tren ini konsisten di semua stasiun, menunjukkan adanya faktor musiman yang memengaruhi kualitas udara, seperti kondisi atmosfer yang stabil dan peningkatan aktivitas pemanasan pada musim dingin.  
   - Wilayah industri dan perkotaan memiliki tingkat PM2.5 lebih tinggi dibandingkan wilayah pinggiran/suburban dan sejarah & budaya, mengindikasikan bahwa aktivitas manusia berperan besar dalam meningkatkan polusi udara.  

2. **Faktor yang Memengaruhi Perubahan Kualitas Udara**  
   - Faktor lingkungan memiliki pengaruh yang bervariasi terhadap konsentrasi PM2.5.  
   - Kecepatan angin (WSPM) memiliki korelasi negatif (-0.22) dengan PM2.5, menunjukkan bahwa angin membantu menyebarkan polutan dan mengurangi konsentrasinya.  
   - Titik embun (DEWP) memiliki korelasi positif (0.20), yang menunjukkan bahwa kelembapan dapat mempengaruhi akumulasi polutan di udara.  
   - Suhu (TEMP) dan tekanan udara (PRES) memiliki korelasi yang sangat rendah dengan PM2.5, menunjukkan bahwa faktor ini tidak berdampak signifikan terhadap polusi udara.  

3. **Sumber Polusi dan Korelasi dengan Polutan Lain**  
   - PM2.5 memiliki korelasi tinggi dengan PM10 (0.78), yang menunjukkan bahwa keduanya kemungkinan berasal dari sumber yang sama, seperti emisi kendaraan dan industri.  
   - CO dan NO2 juga memiliki korelasi sedang dengan PM2.5 (0.62 dan 0.53), menegaskan bahwa aktivitas transportasi berkontribusi besar terhadap polusi udara.  
   - SO2 memiliki korelasi lebih rendah (0.31), yang menunjukkan bahwa pembakaran batu bara dan aktivitas industri masih berpengaruh, tetapi tidak sebesar transportasi.  

4. **Implikasi untuk Kebijakan dan Intervensi**  
   - Peningkatan polusi udara yang terjadi pada musim dingin menunjukkan perlunya kebijakan mitigasi berbasis musim, seperti pembatasan kendaraan atau pengawasan emisi pada periode kritis.  
   - Wilayah industri dan perkotaan memerlukan strategi pengurangan polusi yang lebih agresif dibandingkan wilayah pinggiran dan sejarah, yang secara alami memiliki kualitas udara lebih baik.  
   - Peningkatan penghijauan dan pengaturan zona industri dapat menjadi solusi jangka panjang untuk mengurangi dampak polusi udara di kawasan dengan tingkat PM2.5 tinggi.  

**Rekomendasi**  
- **Pengawasan dan intervensi lebih ketat pada periode kritis**, terutama pada bulan-bulan dengan konsentrasi PM2.5 tertinggi.  
- **Peningkatan kebijakan ramah lingkungan**, terutama di wilayah perkotaan dan industri, untuk mengurangi dampak emisi kendaraan.  
- **Pemantauan dan analisis lanjutan di wilayah dengan konsentrasi PM2.5 tinggi**, guna mengidentifikasi sumber spesifik polusi dan menentukan kebijakan mitigasi yang lebih efektif.  
- **Penerapan sistem peringatan dini** berbasis faktor lingkungan seperti kecepatan angin dan kelembapan, untuk memperkirakan potensi lonjakan polusi udara.  
- **Penyesuaian kebijakan pengendalian emisi** dengan mempertimbangkan faktor lokal dan musim, agar strategi mitigasi lebih tepat sasaran.  